# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [4]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
# !pip install langchain_community

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [19]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=1.2) # Because based on the data, I want the model to be creative and generate diverse responses yet remain coherent.

In [20]:
prompt_str = """Answer the question below using the context:
Context: You are a product expert. You have access to a database of products and their descriptions. Use this information to answer the question.
Question: What is the {product}?
Format: Answer in a single sentence.
Answer: """

#Write a query that would take a variable to describe any product
#https://python.langchain.com/api_reference/langchain/chains/langchain.chains.llm.LLMChain.html
prompt = PromptTemplate(input_variables=["product"], template=prompt_str)

In [ ]:
# chain = LLMChain(llm=llm, prompt=prompt)
chain = prompt | llm | StrOutputParser()

In [23]:
product = "Nintendo Switch 2"  # Example product to query
chain.invoke(product)

'The Nintendo Switch 2 is a rumored next-generation console being developed by Nintendo that is expected to feature upgraded hardware and potentially new capabilities.'

## SimpleSequentialChain

In [31]:
llm2 = ChatOpenAI(temperature=1.6)

# prompt template 1
first_prompt = PromptTemplate(input_variables=["product"], template=prompt_str)

# Chain 1
chain_one = prompt | llm | StrOutputParser()

In [35]:
prompt_str2 = """Answer the question below using the context:
Context: You are a famous video game journalist. You receive the following description : {description}.
Question: Do I need to buy it if I already have a {product2}?
Format: Answer in a single sentence.
Answer: """

# prompt template 2
second_prompt = PromptTemplate(input_variables=["description", "product2"], template=prompt_str2)

def build_input_for_step_two(output_from_step_one):
    return {
        "description": output_from_step_one,
        "product2": "Nintendo Switch Oled"
    }

chain_two = (lambda x: build_input_for_step_two(x)) | second_prompt | llm2 | StrOutputParser()

In [ ]:
output1 = chain_one.invoke({"product": "Nintendo Switch 2"})
output2 = chain_two.invoke(output1)
print(output2)

If you already have a Nintendo Switch Oled and are satisfied with it, there is no need to invest in the Nintendo Switch 2 unless you specifically desire the upgraded features and capabilities it offers.


**Repeat the above twice for different products**

## SequentialChain

In [56]:
llm = ChatOpenAI(temperature=0.6) # Because it's just for review translation and analysis

def build_next_input(prev_output):
    return {"prev": prev_output}

# prompt template 1
first_prompt = PromptTemplate(
  input_variables=["prev"], 
  template=
    """You are a product expert. You are given a product review. 
    Context: {prev}
    Question: What is the product ?
    Format: Answer in an succinct manner.
    Answer:
    """
  )

chain_one = (lambda x: build_next_input(x)) | first_prompt | llm | StrOutputParser()

In [57]:
# prompt template 2
second_prompt = PromptTemplate(
  input_variables=["prev"], 
  template=
    """You are a product expert. You are given a product. 
    Context: {prev}
    Question: What is this product about ?
    Format: Answer in an explanatory manner.
    Answer:
    """
  )

chain_two = (lambda x: build_next_input(x)) | second_prompt | llm | StrOutputParser()

In [58]:
# prompt template 3
third_prompt = PromptTemplate(
  input_variables=["prev"], 
  template=
    """Tu es un traducteur professionnel. 
    Contexte: {prev}
    Question: Tu dois traduire le texte suivant en français.
    Format: Traduis en français.
    Réponse:
    """
  )

chain_three = (lambda x: build_next_input(x)) | third_prompt | llm | StrOutputParser()

In [59]:
llm2 = ChatOpenAI(temperature=1.5) # We want the model to be creative and generate diverse responses yet remain coherent.

# prompt template 4
fourth_prompt = PromptTemplate(
  input_variables=["prev"], 
  template=
    """Tu es un expert produit. 
    Contexte: {prev}
    Question: Est-ce que je dois acheter ce produit ?
    Format: Réponds en expliquant ta réponse.
    Réponse:
    """
  )

chain_four = (lambda x: build_next_input(x)) | fourth_prompt | llm2 | StrOutputParser()

In [61]:
output1 = chain_one.invoke(df['Review'][0])
output2 = chain_two.invoke(output1)
output3 = chain_three.invoke(output2)
output4 = chain_four.invoke(output3)

print("Produit :", output1)
print("Explication :", output2)
print("Summary in French :", output3)
print("Final Recommendation in French :", output4)

Produit : King size sheet set
Explication : A king size sheet set is a bedding product that typically includes a fitted sheet, a flat sheet, and one or two pillowcases, all designed to fit a king size bed. These sheets are larger than standard size sheets, making them ideal for larger mattresses. King size sheet sets come in a variety of materials such as cotton, linen, or microfiber, and can range in thread count for different levels of softness and durability. They are available in a wide range of colors and patterns to match any bedroom decor. A high-quality king size sheet set can provide a comfortable and luxurious sleeping experience, as well as add a stylish touch to your bedroom.
Summary in French : Un ensemble de draps de taille king est un produit de literie qui comprend généralement un drap-housse, un drap plat et une ou deux taies d'oreiller, tous conçus pour s'adapter à un lit king size. Ces draps sont plus grands que les draps de taille standard, ce qui les rend idéaux po

## Router Chain

In [62]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [63]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [64]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [65]:
llm = ChatOpenAI(temperature=0)

In [66]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [67]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [68]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [69]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [70]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/pl/z799lglj00b41cclm26nwxmh0000gn/T/ipykernel_19711/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [71]:
chain.run("What is black body radiation?")

/var/folders/pl/z799lglj00b41cclm26nwxmh0000gn/T/ipykernel_19711/1733274077.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("What is black body radiation?")




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [72]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [73]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This genetic information is crucial for maintaining the integrity and stability of our cells and ultimately our entire body. \n\nAdditionally, DNA is constantly being replicated and repaired in our cells to ensure that genetic information is passed on accurately to the next generation of cells. This process is essential for the growth, development, and overall health of our bodies. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and maintenance of a